## The Ragas Model

In [ ]:
from pydantic import BaseModel, ConfigDict, PrivateAttr, create_model, ValidationError
import typing as t
from datetime import datetime
import inspect

# Type variable for generic models
T = t.TypeVar("T")


# Base metadata class for Notion field types
class NotionFieldMeta:
    """Base metadata class for Notion field types."""

    NOTION_FIELD_TYPE: t.ClassVar[str] = ""
    required: bool = True
    name: str = ""
    _instance_cache = {}  # Global instance cache

    def __class_getitem__(cls, params):
        """Support for Class[field_type] syntax"""
        return cls

    def __new__(cls, required=True):
        """Create a new instance or return a cached one"""
        if cls not in NotionFieldMeta._instance_cache:
            NotionFieldMeta._instance_cache[cls] = {}

        key = (cls, required)
        if key not in NotionFieldMeta._instance_cache[cls]:
            instance = super().__new__(cls)
            instance.required = required
            NotionFieldMeta._instance_cache[cls][key] = instance

        return NotionFieldMeta._instance_cache[cls][key]

    def __call__(self, required=True):
        """Support both Class and Class() syntax"""
        return self.__class__(required=required)

    def validate(self, value: t.Any) -> t.Any:
        """Validate field value."""
        if value is None and self.required:
            raise ValueError(f"Field {self.name} is required")
        return value

    def to_notion(self, value: t.Any) -> dict:
        """Convert Python value to Notion format."""
        raise NotImplementedError()

    def from_notion(self, data: dict) -> t.Any:
        """Convert Notion format to Python value."""
        raise NotImplementedError()

    def to_notion_property(self) -> dict:
        """Define Notion property schema for database creation."""
        return {self.name: {"type": self.NOTION_FIELD_TYPE, self.NOTION_FIELD_TYPE: {}}}


# Implementation of field types
class ID(NotionFieldMeta):
    """Notion ID field type."""

    NOTION_FIELD_TYPE = "unique_id"

    def validate(self, value: t.Any) -> t.Optional[int]:
        value = super().validate(value)
        if value is not None and not isinstance(value, int):
            raise ValueError(f"ID must be an integer, got {type(value)}")
        return value

    def to_notion(self, value: int) -> dict:
        return {self.name: {"type": "unique_id", "unique_id": value}}

    def from_notion(self, data: dict) -> t.Optional[int]:
        if "properties" in data and self.name in data["poperties"]:
            unique_id = data["properties"][self.name]["unique_id"]
            # Handle both integer values and structured API responses
            if isinstance(unique_id, int):
                return unique_id
            elif isinstance(unique_id, dict) and "number" in unique_id:
                return unique_id["number"]
        elif self.name in data:
            unique_id = data[self.name]["unique_id"]
            # Handle both integer values and structured API responses
            if isinstance(unique_id, int):
                return unique_id
            elif isinstance(unique_id, dict) and "number" in unique_id:
                return unique_id["number"]
            return None

    def to_notion_property(self) -> dict:
        return {self.name: {"type": "unique_id", "unique_id": {"prefix": None}}}


class Text(NotionFieldMeta):
    """Notion rich text field type."""

    NOTION_FIELD_TYPE = "rich_text"
    CHUNK_SIZE = 2000  # Notion's character limit per rich text block

    def validate(self, value: t.Any) -> t.Optional[str]:
        value = super().validate(value)
        if value is not None and not isinstance(value, str):
            raise ValueError(f"Text must be a string, got {type(value)}")
        return value

    def to_notion(self, value: str) -> dict:
        if not value:
            return {self.name: {self.NOTION_FIELD_TYPE: []}}

        chunks = [
            value[i : i + self.CHUNK_SIZE]
            for i in range(0, len(value), self.CHUNK_SIZE)
        ]
        rich_text_array = [{"text": {"content": chunk}} for chunk in chunks]

        return {self.name: {self.NOTION_FIELD_TYPE: rich_text_array}}

    def from_notion(self, data: dict) -> t.Optional[str]:
        if "properties" in data and self.name in data["properties"]:
            rich_text = data["properties"][self.name][self.NOTION_FIELD_TYPE]
        elif self.name in data:
            rich_text = data[self.name][self.NOTION_FIELD_TYPE]
        else:
            return None

        if not rich_text:
            return None

        return "".join(item["text"]["content"] for item in rich_text if "text" in item)


class Select(NotionFieldMeta):
    """Notion select field type."""

    NOTION_FIELD_TYPE = "select"
    options: list = []

    def __new__(cls, options=None, required=True):
        """Create a new instance or return a cached one"""
        if cls not in NotionFieldMeta._instance_cache:
            NotionFieldMeta._instance_cache[cls] = {}

        key = (cls, tuple(options) if options else None, required)
        if key not in NotionFieldMeta._instance_cache[cls]:
            instance = super(NotionFieldMeta, cls).__new__(cls)
            instance.required = required
            instance.options = options
            NotionFieldMeta._instance_cache[cls][key] = instance

        return NotionFieldMeta._instance_cache[cls][key]

    def __init__(self, options=None, required=True):
        self.required = required
        self.options = options

    def validate(self, value: t.Any) -> t.Optional[str]:
        value = super().validate(value)
        if value is not None:
            if not isinstance(value, str):
                raise ValueError(f"Select value must be a string, got {type(value)}")

            # Check options or extract from Literal type
            options = self.options
            if not options:
                # We'll extract options later when we have field info
                pass
            elif options and value not in options:
                raise ValueError(
                    f"Value '{value}' is not in allowed options: {options}"
                )
        return value

    def to_notion(self, value: str) -> dict:
        if not value:
            return {self.name: {self.NOTION_FIELD_TYPE: None}}
        return {self.name: {self.NOTION_FIELD_TYPE: {"name": value}}}

    def from_notion(self, data: dict) -> t.Optional[str]:
        if "properties" in data and self.name in data["properties"]:
            select_data = data["properties"][self.name][self.NOTION_FIELD_TYPE]
        elif self.name in data:
            select_data = data[self.name][self.NOTION_FIELD_TYPE]
        else:
            return None

        if not select_data:
            return None

        return select_data["name"]

    def to_notion_property(self) -> dict:
        options = [{"name": option} for option in (self.options or [])]
        return {self.name: {"type": "select", "select": {"options": options}}}


# Convenience alias
Title = Text  # For clarity when using as a title


# The main Notion model base class
class NotionModel(BaseModel):
    """Base model for Notion integration with Pydantic."""

    model_config = ConfigDict(extra="forbid", arbitrary_types_allowed=True)

    # Notion metadata stored as private attributes
    _page_id: t.Optional[str] = PrivateAttr(default=None)
    _created_time: t.Optional[datetime] = PrivateAttr(default=None)
    _last_edited_time: t.Optional[datetime] = PrivateAttr(default=None)

    def __init__(self, **data):
        # Extract Notion metadata before validation
        page_id = data.pop("page_id", None)
        created_time = data.pop("created_time", None)
        last_edited_time = data.pop("last_edited_time", None)

        # Process field metadata
        self._process_field_metadata()

        # Pre-validate field data against Notion field metadata
        self._validate_notion_fields(data)

        # Initialize using Pydantic
        super().__init__(**data)

        # Set private attributes after initialization
        self._page_id = page_id
        self._created_time = created_time
        self._last_edited_time = last_edited_time

    def _process_field_metadata(self):
        """Process field metadata to handle class references and Literal types."""
        for field_name, field_info in self.model_fields.items():
            updated_metadata = list(field_info.metadata)
            for i, meta in enumerate(updated_metadata):
                # If the metadata is a NotionFieldMeta class (not instance), instantiate it
                if inspect.isclass(meta) and issubclass(meta, NotionFieldMeta):
                    updated_metadata[i] = meta()

                # Set field name for the metadata
                if isinstance(meta, NotionFieldMeta):
                    meta.name = field_name

                    # Handle Select fields with Literal type
                    if isinstance(meta, Select) and not meta.options:
                        origin = t.get_origin(field_info.annotation)
                        if origin is t.Annotated:
                            # Get the real type from Annotated
                            real_type = t.get_args(field_info.annotation)[0]
                            origin = t.get_origin(real_type)
                            if origin is t.Literal:
                                meta.options = list(t.get_args(real_type))

            # Update the field metadata
            field_info.metadata = tuple(updated_metadata)

    def _validate_notion_fields(self, data: dict):
        """Pre-validate data against Notion field metadata."""
        errors = []

        for field_name, field_info in self.model_fields.items():
            if field_name in data:
                value = data[field_name]

                for meta in field_info.metadata:
                    if isinstance(meta, NotionFieldMeta):
                        try:
                            meta.validate(value)
                        except ValueError as e:
                            errors.append(
                                {
                                    "loc": (field_name,),
                                    "msg": str(e),
                                    "type": "value_error",
                                    "input": value,
                                }
                            )

        if errors:
            raise ValidationError.from_exception_data(
                title=f"Validation error for {self.__class__.__name__}",
                line_errors=errors,
            )

    @property
    def page_id(self) -> t.Optional[str]:
        """Get the Notion page ID."""
        return self._page_id

    def to_notion(self) -> dict:
        """Convert model to Notion format."""
        result = {"properties": {}}

        for field_name, field_info in self.model_fields.items():
            for meta in field_info.metadata:
                if isinstance(meta, NotionFieldMeta):
                    value = getattr(self, field_name)
                    if value is not None:  # Skip None values
                        notion_data = meta.to_notion(value)
                        result["properties"].update(notion_data)

        # Add page_id if it exists
        if self._page_id:
            result["id"] = self._page_id

        return result

    @classmethod
    def from_notion(cls, data: dict):
        """Create model instance from Notion data."""
        values = {}

        # Process fields based on metadata
        for field_name, field_info in cls.model_fields.items():
            for meta in field_info.metadata:
                if isinstance(meta, NotionFieldMeta):
                    meta.name = field_name  # Ensure name is set
                    value = meta.from_notion(data)
                    if value is not None:
                        values[field_name] = value

        # Include Notion metadata
        notion_metadata = {}
        if "id" in data:
            notion_metadata["page_id"] = data["id"]
        if "created_time" in data:
            notion_metadata["created_time"] = data["created_time"]
        if "last_edited_time" in data:
            notion_metadata["last_edited_time"] = data["last_edited_time"]

        # Create instance
        instance = cls(**values)

        # Set private attributes
        if "page_id" in notion_metadata:
            instance._page_id = notion_metadata["page_id"]
        if "created_time" in notion_metadata:
            instance._created_time = notion_metadata["created_time"]
        if "last_edited_time" in notion_metadata:
            instance._last_edited_time = notion_metadata["last_edited_time"]

        return instance

    def get_notion_properties(self) -> dict:
        """Get Notion property definitions for creating database."""
        properties = {}

        for field_name, field_info in self.model_fields.items():
            for meta in field_info.metadata:
                if isinstance(meta, NotionFieldMeta):
                    meta.name = field_name
                    properties.update(meta.to_notion_property())

        return properties

In [ ]:
# Example usage
class SupermeDataset(NotionModel):
    id: t.Annotated[int, ID]
    query: t.Annotated[str, Title]
    persona: t.Annotated[t.Literal["user", "agent"], Select]

In [ ]:
# This should work
ds1 = SupermeDataset(id=1, query="test query", persona="user")
print(f"Valid instance created: {ds1.model_dump_json()}")

# This should fail validation with an error
try:
    ds2 = SupermeDataset(id=1, query="test query", persona="invalid")
    print("This should not print!")
except ValidationError as e:
    print(f"Validation failed as expected: {e}")

# Convert to Notion format
notion_data = ds1.to_notion()
print(f"Notion format: {notion_data}")

# Create from Notion data
notion_response = {
    "id": "page_123",
    "properties": {
        "id": {"type": "unique_id", "unique_id": 42},
        "query": {"rich_text": [{"text": {"content": "from notion"}}]},
        "persona": {"select": {"name": "agent"}},
    },
    "created_time": "2023-01-01T00:00:00Z",
    "last_edited_time": "2023-01-02T00:00:00Z",
}

ds_from_notion = SupermeDataset.from_notion(notion_response)
print(f"From Notion: {ds_from_notion.model_dump_json()}")
print(f"Page ID: {ds_from_notion.page_id}")

Valid instance created: {"id":1,"query":"test query","persona":"user"}
Validation failed as expected: 1 validation error for SupermeDataset
persona
  Input should be 'user' or 'agent' [type=literal_error, input_value='invalid', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
Notion format: {'properties': {'id': {'type': 'unique_id', 'unique_id': 1}, 'query': {'rich_text': [{'text': {'content': 'test query'}}]}, 'persona': {'select': {'name': 'user'}}}}


KeyError: 'poperties'

## Building the API Endpoint

In [ ]:
RAGAS_APP_TOKEN = "apt.47bd-c55e4a45b27c-02f8-8446-1441f09b-651a8"
RAGAS_API_ENDPOINT = "http://localhost:8087"

In [ ]:
import httpx
import asyncio
import typing as t


class RelayClient:
    """Async client for the Relay API."""

    def __init__(self, base_url: str, app_token: t.Optional[str] = None):
        if not app_token:
            raise ValueError("app_token must be provided")

        self.base_url = f"{base_url.rstrip('/')}/api/v1"
        self.app_token = app_token

    async def _request(
        self,
        method: str,
        endpoint: str,
        params: t.Optional[t.Dict] = None,
        json_data: t.Optional[t.Dict] = None,
    ) -> t.Dict:
        ""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = {}
        headers["X-App-Token"] = self.app_token

        async with httpx.AsyncClient() as client:
            response = await client.request(
                method=method, url=url, params=params, json=json_data, headers=headers
            )

            data = response.json()

            if response.status_code >= 400 or data.get("status") == "error":
                error_msg = data.get("message", "Unknown error")
                raise Exception(f"API Error ({response.status_code}): {error_msg}")

            return data.get("data")

    async def list_projects(
        self,
        ids: t.Optional[t.List[str]] = None,
        limit: int = 50,
        offset: int = 0,
        order_by: t.Optional[str] = None,
        sort_dir: t.Optional[str] = None,
    ) -> t.Dict:
        params = {"limit": limit, "offset": offset}

        if ids:
            params["ids"] = ",".join(ids)

        if order_by:
            params["order_by"] = order_by

        if sort_dir:
            params["sort_dir"] = sort_dir

        return await self._request("GET", "projects", params=params)

In [ ]:
# Initialize client with your authentication token
client = RelayClient(base_url=RAGAS_API_ENDPOINT, app_token=RAGAS_APP_TOKEN)

# List projects
try:
    projects = await client.list_projects(limit=10)
    print(f"Found {len(projects)} projects:")
    for project in projects:
        print(f"- {project['title']} (ID: {project['id']})")
except Exception as e:
    print(f"Error: {e}")

Found 2 projects:
Error: string indices must be integers, not 'str'


In [ ]:
from fastcore.utils import patch

### Projects

In [ ]:
@patch
async def get_project(self: RelayClient, project_id: str) -> t.Dict:
    """Get a specific project by ID."""
    return await self._request("GET", f"projects/{project_id}")


@patch
async def create_project(
    self: RelayClient, title: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new project."""
    data = {"title": title}
    if description:
        data["description"] = description
    return await self._request("POST", "projects", json_data=data)


@patch
async def update_project(
    self: RelayClient,
    project_id: str,
    title: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing project."""
    data = {}
    if title:
        data["title"] = title
    if description:
        data["description"] = description
    return await self._request("PATCH", f"projects/{project_id}", json_data=data)


@patch
async def delete_project(self: RelayClient, project_id: str) -> None:
    """Delete a project."""
    await self._request("DELETE", f"projects/{project_id}")

In [ ]:
await client.create_project("test project", "test description")

{'id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6',
 'title': 'test project',
 'description': 'test description',
 'created_at': '2025-03-30T02:33:38.751793+00:00'}

In [ ]:
await client.list_projects()

{'items': [{'id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6',
   'title': 'test project',
   'description': 'test description',
   'created_at': '2025-03-30T02:33:38.751793+00:00'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 1,
  'order_by': 'created_at',
  'sort_dir': 'desc'}}

In [ ]:
TEST_PROJECT_ID = "e1b3f1e4-d344-48f4-a178-84e7e32e6ab6"
project = await client.get_project(TEST_PROJECT_ID)

### Datasets

In [ ]:
@patch
async def list_datasets(
    self: RelayClient,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List datasets in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request("GET", f"projects/{project_id}/datasets", params=params)


@patch
async def get_dataset(self: RelayClient, project_id: str, dataset_id: str) -> t.Dict:
    """Get a specific dataset."""
    return await self._request("GET", f"projects/{project_id}/datasets/{dataset_id}")


@patch
async def create_dataset(
    self: RelayClient, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new dataset in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._request(
        "POST", f"projects/{project_id}/datasets", json_data=data
    )


@patch
async def update_dataset(
    self: RelayClient,
    project_id: str,
    dataset_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing dataset."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._request(
        "PATCH", f"projects/{project_id}/datasets/{dataset_id}", json_data=data
    )


@patch
async def delete_dataset(self: RelayClient, project_id: str, dataset_id: str) -> None:
    """Delete a dataset."""
    await self._request("DELETE", f"projects/{project_id}/datasets/{dataset_id}")

In [ ]:
# check project ID
projects = await client.list_projects()
projects["items"][0]["id"], TEST_PROJECT_ID

('e1b3f1e4-d344-48f4-a178-84e7e32e6ab6',
 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6')

In [ ]:
# Create a new dataset
new_dataset = await client.create_dataset(
    projects["items"][0]["id"], "New Dataset", "This is a new dataset"
)
print(f"New dataset created: {new_dataset}")

New dataset created: {'id': 'dc772b6e-2691-44b9-a6aa-9a0099a37346', 'name': 'New Dataset', 'description': 'This is a new dataset', 'created_at': '2025-03-30T02:35:37.606287+00:00', 'version_counter': 0, 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}


In [ ]:
# List datasets in the project
datasets = await client.list_datasets(projects["items"][0]["id"])
print(f"Found {len(datasets)} datasets")

Found 2 datasets


In [ ]:
updated_dataset = await client.update_dataset(
    projects["items"][0]["id"],
    datasets["items"][0]["id"],
    "Updated Dataset",
    "This is an updated dataset",
)
print(f"Updated dataset: {updated_dataset}")

Updated dataset: {'id': 'dc772b6e-2691-44b9-a6aa-9a0099a37346', 'name': 'Updated Dataset', 'description': 'This is an updated dataset', 'created_at': '2025-03-30T02:35:37.606287+00:00', 'version_counter': 0, 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}


In [ ]:
# Delete the dataset
await client.delete_dataset(projects["items"][0]["id"], datasets["items"][0]["id"])
print("Dataset deleted")

Dataset deleted


### Experiments

In [ ]:
@patch
async def list_experiments(
    self: RelayClient,
    project_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List experiments in a project."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request(
        "GET", f"projects/{project_id}/experiments", params=params
    )


@patch
async def get_experiment(
    self: RelayClient, project_id: str, experiment_id: str
) -> t.Dict:
    """Get a specific experiment."""
    return await self._request(
        "GET", f"projects/{project_id}/experiments/{experiment_id}"
    )


@patch
async def create_experiment(
    self: RelayClient, project_id: str, name: str, description: t.Optional[str] = None
) -> t.Dict:
    """Create a new experiment in a project."""
    data = {"name": name}
    if description:
        data["description"] = description
    return await self._request(
        "POST", f"projects/{project_id}/experiments", json_data=data
    )


@patch
async def update_experiment(
    self: RelayClient,
    project_id: str,
    experiment_id: str,
    name: t.Optional[str] = None,
    description: t.Optional[str] = None,
) -> t.Dict:
    """Update an existing experiment."""
    data = {}
    if name:
        data["name"] = name
    if description:
        data["description"] = description
    return await self._request(
        "PATCH", f"projects/{project_id}/experiments/{experiment_id}", json_data=data
    )


@patch
async def delete_experiment(
    self: RelayClient, project_id: str, experiment_id: str
) -> None:
    """Delete an experiment."""
    await self._request("DELETE", f"projects/{project_id}/experiments/{experiment_id}")

In [ ]:
# create a new experiment
new_experiment = await client.create_experiment(
    projects["items"][0]["id"], "New Experiment", "This is a new experiment"
)
print(f"New experiment created: {new_experiment}")
# list experiments
experiments = await client.list_experiments(projects["items"][0]["id"])
print(f"Found {len(experiments)} experiments")
# get a specific experiment
experiment = await client.get_experiment(
    projects["items"][0]["id"], experiments["items"][0]["id"]
)
print(f"Experiment: {experiment}")
# update an experiment
updated_experiment = await client.update_experiment(
    projects["items"][0]["id"],
    experiments["items"][0]["id"],
    "Updated Experiment",
    "This is an updated experiment",
)
print(f"Updated experiment: {updated_experiment}")
# delete an experiment
await client.delete_experiment(projects["items"][0]["id"], experiments["items"][0]["id"])
print("Experiment deleted")

New experiment created: {'id': '17097da4-29e6-4b94-b447-87ef00c21343', 'name': 'New Experiment', 'description': 'This is a new experiment', 'created_at': '2025-03-30T03:03:13.934694+00:00', 'version_counter': 0, 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}
Found 2 experiments
Experiment: {'id': '17097da4-29e6-4b94-b447-87ef00c21343', 'name': 'New Experiment', 'description': 'This is a new experiment', 'created_at': '2025-03-30T03:03:13.934694+00:00', 'version_counter': 0, 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}
Updated experiment: {'id': '17097da4-29e6-4b94-b447-87ef00c21343', 'name': 'Updated Experiment', 'description': 'This is an updated experiment', 'created_at': '2025-03-30T03:03:13.934694+00:00', 'version_counter': 0, 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}
Experiment deleted


In [ ]:
await client.list_experiments(TEST_PROJECT_ID)

{'items': [],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 0,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

### Columns (for datasets)

The API supports the following column types:

- `number`: Numeric values
- `longText`: Text content
- `select`: Single selection from predefined options
- `date`: Date values
- `multiSelect`: Multiple selections from predefined options
- `checkbox`: Boolean values
- `custom`: Custom column types with specific behavior

Each column type has specific settings that can be configured through the `settings` object.

In [ ]:
from enum import StrEnum


class ColumnType(StrEnum):
    NUMBER = "number"
    TEXT = "text"
    LONG_TEXT = "longText"
    SELECT = "select"
    DATE = "date"
    MULTI_SELECT = "multiSelect"
    CHECKBOX = "checkbox"
    CUSTOM = "custom"

In [ ]:
@patch
async def list_dataset_columns(
    self: RelayClient,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request(
        "GET", f"projects/{project_id}/datasets/{dataset_id}/columns", params=params
    )


@patch
async def get_dataset_column(
    self: RelayClient, project_id: str, dataset_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in a dataset."""
    return await self._request(
        "GET", f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )


@patch
async def create_dataset_column(
    self: RelayClient,
    project_id: str,
    dataset_id: str,
    id: str,
    name: str,
    type: str,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """
    Create a new column in a dataset.

    Raises:
        RelayAPIError: If column ID already exists (409 Conflict) or other API errors
    """
    settings = settings or {}
    data = {"id": id, "name": name, "col_order": col_order, "type": type, "settings": settings}
    return await self._request(
        "POST", f"projects/{project_id}/datasets/{dataset_id}/columns", json_data=data
    )


@patch
async def update_dataset_column(
    self: RelayClient, project_id: str, dataset_id: str, column_id: str, **column_data
) -> t.Dict:
    """Update an existing column in a dataset."""
    return await self._request(
        "PATCH",
        f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}",
        json_data=column_data,
    )


@patch
async def delete_dataset_column(
    self: RelayClient, project_id: str, dataset_id: str, column_id: str
) -> None:
    """Delete a column from a dataset."""
    await self._request(
        "DELETE", f"projects/{project_id}/datasets/{dataset_id}/columns/{column_id}"
    )

In [ ]:
datasets = await client.create_dataset(
    projects["items"][0]["id"],
    "New Dataset for testing columns",
    "This is a new dataset for testing columns",
)
datasets

{'id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8',
 'name': 'New Dataset for testing columns',
 'description': 'This is a new dataset for testing columns',
 'created_at': '2025-03-30T03:04:06.619835+00:00',
 'version_counter': 0,
 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}

In [ ]:
# add a new column to the dataset
new_column = await client.create_dataset_column(
    project_id=projects["items"][0]["id"],
    dataset_id=datasets["id"],
    id="new_column_3",
    name="New Column 3",
    type=ColumnType.TEXT.value,
    settings={
        "max_length": 255,
        "is_required": True,
    },
)
new_column

{'id': 'new_column_3',
 'name': 'New Column 3',
 'type': 'longText',
 'settings': {'max_length': 255, 'is_required': True},
 'created_at': '2025-03-30T03:04:10.937689+00:00',
 'updated_at': '2025-03-30T03:04:10.937689+00:00',
 'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}

In [ ]:
await client.list_dataset_columns(projects["items"][0]["id"], datasets["id"])

{'items': [{'id': 'new_column_3',
   'name': 'New Column 3',
   'type': 'longText',
   'settings': {'max_length': 255, 'is_required': True},
   'created_at': '2025-03-30T03:04:10.937689+00:00',
   'updated_at': '2025-03-30T03:04:10.937689+00:00',
   'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 1,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [ ]:
col3 = await client.get_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)
col3

{'id': 'new_column_3',
 'name': 'New Column 3',
 'type': 'longText',
 'settings': {'max_length': 255, 'is_required': True},
 'created_at': '2025-03-30T03:04:10.937689+00:00',
 'updated_at': '2025-03-30T03:04:10.937689+00:00',
 'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}

In [ ]:
await client.update_dataset_column(
    projects["items"][0]["id"],
    datasets["id"],
    "new_column_3",
    name="New Column 3 Updated",
    type=ColumnType.NUMBER.value,
)

{'id': 'new_column_3',
 'name': 'New Column 3 Updated',
 'type': 'number',
 'settings': {'max_length': 255, 'is_required': True},
 'created_at': '2025-03-30T03:04:10.937689+00:00',
 'updated_at': '2025-03-30T03:04:33.47167+00:00',
 'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}

In [ ]:
await client.delete_dataset_column(
    projects["items"][0]["id"], datasets["id"], "new_column_3"
)

### Rows (for datasets)

In [ ]:
@patch
async def list_dataset_rows(
    self: RelayClient,
    project_id: str,
    dataset_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List rows in a dataset."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request(
        "GET", f"projects/{project_id}/datasets/{dataset_id}/rows", params=params
    )


@patch
async def get_dataset_row(
    self: RelayClient, project_id: str, dataset_id: str, row_id: str
) -> t.Dict:
    """Get a specific row in a dataset."""
    return await self._request(
        "GET", f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )


@patch
async def create_dataset_row(
    self: RelayClient, project_id: str, dataset_id: str, id: str, data: t.Dict
) -> t.Dict:
    """Create a new row in a dataset."""
    row_data = {"id": id, "data": data}
    return await self._request(
        "POST", f"projects/{project_id}/datasets/{dataset_id}/rows", json_data=row_data
    )


@patch
async def update_dataset_row(
    self: RelayClient, project_id: str, dataset_id: str, row_id: str, data: t.Dict
) -> t.Dict:
    """Update an existing row in a dataset."""
    row_data = {"data": data}
    return await self._request(
        "PATCH",
        f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}",
        json_data=row_data,
    )


@patch
async def delete_dataset_row(
    self: RelayClient, project_id: str, dataset_id: str, row_id: str
) -> None:
    """Delete a row from a dataset."""
    await self._request(
        "DELETE", f"projects/{project_id}/datasets/{dataset_id}/rows/{row_id}"
    )

In [ ]:
await client.create_dataset_row(
    projects["items"][0]["id"],
    datasets["id"],
    "1",
    {"name": "New Row 1", "age": 30},
)


{'id': '1',
 'data': {'age': 30, 'name': 'New Row 1'},
 'created_at': '2025-03-30T03:05:22.502878+00:00',
 'updated_at': '2025-03-30T03:05:22.502878+00:00',
 'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}

In [ ]:
await client.list_dataset_rows(projects["items"][0]["id"], datasets["id"])

{'items': [{'id': '1',
   'data': {'age': 30, 'name': 'New Row 1'},
   'created_at': '2025-03-30T03:05:22.502878+00:00',
   'updated_at': '2025-03-30T03:05:22.502878+00:00',
   'datatable_id': 'f494bc5a-c80e-4fba-b0ec-a22ad3b4e9d8'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 1,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

In [ ]:
import uuid
import string

def create_nano_id(size=12):
    # Define characters to use (alphanumeric)
    alphabet = string.ascii_letters + string.digits
    
    # Generate UUID and convert to int
    uuid_int = uuid.uuid4().int
    
    # Convert to base62
    result = ""
    while uuid_int:
        uuid_int, remainder = divmod(uuid_int, len(alphabet))
        result = alphabet[remainder] + result
    
    # Pad if necessary and return desired length
    return result[:size]

# Usage
nano_id = create_nano_id()  # e.g., "8dK9cNw3mP5x"

In [ ]:
# Nano ID is generally faster
from nanoid import generate
from uuid import uuid4
import timeit

def gen_nano():
    return generate()

def gen_uuid():
    return str(uuid4())

# Compare performance
nano_time = timeit.timeit(gen_nano, number=10000)
uuid_time = timeit.timeit(gen_uuid, number=10000)
custom_nano_time = timeit.timeit(create_nano_id, number=10000)

In [ ]:
nano_time, uuid_time, custom_nano_time

(0.02608395798597485, 0.019044874992687255, 0.02830641600303352)

### Get a Dataset Visualized - Created From UI
Lets Create a new dataset and add columns and rows via the endpoint to see how it behaves

In [ ]:
# generate a dataset
dataset = await client.create_dataset(
    project_id=TEST_PROJECT_ID,
    name="Dataset Visualized from UI",
    description="This is a dataset created from the UI",
)

# show url
WEB_ENDPOINT = "http://localhost:3000"
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'http://localhost:3000/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/e2366381-3989-4154-ac91-5a1e75d18bab'

In [ ]:
# list columns
columns = await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])
# list rows
rows = await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])


In [ ]:
columns


{'items': [{'id': 'uujCCGzIBwQ2D0K4DL5si',
   'name': 'id',
   'type': 'number',
   'settings': {'id': 'uujCCGzIBwQ2D0K4DL5si',
    'name': 'id',
    'type': 'number',
    'width': 192,
    'position': 0,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-03-30T16:24:11.001641+00:00',
   'updated_at': '2025-03-30T16:24:20.17409+00:00',
   'datatable_id': 'e2366381-3989-4154-ac91-5a1e75d18bab'},
  {'id': 'HnwarbjjKuEPyZropCJdG',
   'name': 'query',
   'type': 'text',
   'settings': {'id': 'HnwarbjjKuEPyZropCJdG',
    'name': 'query',
    'type': 'text',
    'width': 192,
    'position': 1,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-03-30T16:24:20.327007+00:00',
   'updated_at': '2025-03-30T16:24:29.020552+00:00',
   'datatable_id': 'e2366381-3989-4154-ac91-5a1e75d18bab'},
  {'id': 'q-iulmBa5tJfewVLmY8HR',
   'name': 'persona',
   'type': 'select',
   'settings': {'id': 'q-iulmBa5tJfewVLmY8HR',
    'name': 'persona',
    'type': 'select',
 

In [ ]:
rows

{'items': [{'id': 'l2t_eNfBJEq713jnolcF-',
   'data': {'id': 'l2t_eNfBJEq713jnolcF-',
    'HnwarbjjKuEPyZropCJdG': 'This is a test',
    'q-iulmBa5tJfewVLmY8HR': 'jithin',
    'uujCCGzIBwQ2D0K4DL5si': '1'},
   'created_at': '2025-03-30T16:29:27.007777+00:00',
   'updated_at': '2025-03-30T16:29:43.17193+00:00',
   'datatable_id': 'e2366381-3989-4154-ac91-5a1e75d18bab'},
  {'id': 'tmXwh5ePIIPSYfWy6mQHU',
   'data': {'id': 'tmXwh5ePIIPSYfWy6mQHU',
    'HnwarbjjKuEPyZropCJdG': 'This is another test',
    'q-iulmBa5tJfewVLmY8HR': 'ganesh',
    'uujCCGzIBwQ2D0K4DL5si': '2'},
   'created_at': '2025-03-30T16:29:43.628186+00:00',
   'updated_at': '2025-03-30T16:29:59.193342+00:00',
   'datatable_id': 'e2366381-3989-4154-ac91-5a1e75d18bab'}],
 'pagination': {'offset': 0,
  'limit': 50,
  'total': 2,
  'order_by': 'created_at',
  'sort_dir': 'asc'}}

### Get a Dataset Visualized - from API

we want to be able to use the API with python data like this `t.List[t.Dict]`.
```py
# how we want the data to look
data = [
    {
        "id": "1",
        "query": "What is the capital of France?",
        "persona": "John",
        "ground_truth": "Paris",
    },
    {
        "id": "2",
        "query": "What is the capital of Germany?",
        "persona": "Jane",
        "ground_truth": "Berlin",
    },
    {
        "id": "3",
        "query": "What is the capital of Italy?",
        "persona": "John",
        "ground_truth": "Rome",
    },
]
```

In [ ]:
# data
columns = {
    "id": {"name": "id", "type": ColumnType.NUMBER.value, "id": create_nano_id()},
    "query": {"name": "query", "type": ColumnType.TEXT.value, "id": create_nano_id()},
    "persona": {"name": "persona", "type": ColumnType.SELECT.value, "id": create_nano_id()},
    "ground_truth": {"name": "ground_truth", "type": ColumnType.TEXT.value, "id": create_nano_id()},
}

rows = [{
    "id": {"data": "1", "column_id": columns["id"]["id"]},
    "query": {"data": "What is the capital of France?", "column_id": columns["query"]["id"]},
    "persona": {"data": "John", "column_id": columns["persona"]["id"]},
    "ground_truth": {"data": "Paris", "column_id": columns["ground_truth"]["id"]},
}, {
    "id": {"data": "2", "column_id": columns["id"]["id"]},
    "query": {"data": "What is the capital of Germany?", "column_id": columns["query"]["id"]},
    "persona": {"data": "Jane", "column_id": columns["persona"]["id"]},
    "ground_truth": {"data": "Berlin", "column_id": columns["ground_truth"]["id"]},
}, {
    "id": {"data": "3", "column_id": columns["id"]["id"]},
    "query": {"data": "What is the capital of Italy?", "column_id": columns["query"]["id"]},
    "persona": {"data": "John", "column_id": columns["persona"]["id"]},
    "ground_truth": {"data": "Rome", "column_id": columns["ground_truth"]["id"]},
}]



In [ ]:
DEFAULT_SETTINGS = {
    "width": 100,
    "isVisible": True,
    "isEditable": True,
}
# create new dataset
dataset = await client.create_dataset(
    project_id=TEST_PROJECT_ID,
    name="Dataset Visualized from API",
    description="This is a dataset created from the API",
)


In [ ]:
# get dataset url
url = f"{WEB_ENDPOINT}/dashboard/projects/{TEST_PROJECT_ID}/datasets/{dataset['id']}"
url

'http://localhost:3000/dashboard/projects/e1b3f1e4-d344-48f4-a178-84e7e32e6ab6/datasets/ddebc6ff-95c1-488b-91ca-b29adbe59a6c'

In [ ]:
# delete dataset
await client.delete_dataset(TEST_PROJECT_ID, dataset["id"])

In [ ]:
for col in columns.values():
    # copy default settings and add things
    await client.create_dataset_column(
        project_id=TEST_PROJECT_ID,
        dataset_id=dataset["id"],
        id=col["id"],
        name=col["name"],
        type=col["type"],
        settings=DEFAULT_SETTINGS,
)

In [ ]:
# list columns
await client.list_dataset_columns(TEST_PROJECT_ID, dataset["id"])

{'items': [{'id': 'gFJjoJ2m3uhZ',
   'name': 'id',
   'type': 'number',
   'settings': {'id': 'gFJjoJ2m3uhZ',
    'name': 'id',
    'type': 'number',
    'width': 100,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-03-30T19:23:39.650696+00:00',
   'updated_at': '2025-03-30T19:23:39.650696+00:00',
   'datatable_id': 'ddebc6ff-95c1-488b-91ca-b29adbe59a6c'},
  {'id': 'eT0gFBhCCYIQ',
   'name': 'query',
   'type': 'text',
   'settings': {'id': 'eT0gFBhCCYIQ',
    'name': 'query',
    'type': 'text',
    'width': 100,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-03-30T19:23:40.627392+00:00',
   'updated_at': '2025-03-30T19:23:40.627392+00:00',
   'datatable_id': 'ddebc6ff-95c1-488b-91ca-b29adbe59a6c'},
  {'id': 'gk8SrPiAqlFm',
   'name': 'persona',
   'type': 'select',
   'settings': {'id': 'gk8SrPiAqlFm',
    'name': 'persona',
    'type': 'select',
    'width': 100,
    'isVisible': True,
    'isEditable': True},
   'created_at': '2025-03-

In [ ]:
# add rows
for row in rows:
    row_data = {}
    # craft row data
    for col in row.values():
        row_data[col["column_id"]] = col["data"]
    await client.create_dataset_row(
        project_id=TEST_PROJECT_ID,
        dataset_id=dataset["id"],
        id=create_nano_id(),
        data=row_data
    )

In [ ]:
# list rows
await client.list_dataset_rows(TEST_PROJECT_ID, dataset["id"])

{'items': [{'id': 'fkDyIAn6OVtW',
   'data': {'id': 'fkDyIAn6OVtW',
    'cCnutJRZqI9J': 'Paris',
    'eT0gFBhCCYIQ': 'What is the capital of France?',
    'gFJjoJ2m3uhZ': '1',
    'gk8SrPiAqlFm': 'John'},
   'created_at': '2025-03-30T19:31:43.850379+00:00',
   'updated_at': '2025-03-30T19:31:43.850379+00:00',
   'datatable_id': 'ddebc6ff-95c1-488b-91ca-b29adbe59a6c'},
  {'id': 'bGMquZi0i0B6',
   'data': {'id': 'bGMquZi0i0B6',
    'cCnutJRZqI9J': 'Berlin',
    'eT0gFBhCCYIQ': 'What is the capital of Germany?',
    'gFJjoJ2m3uhZ': '2',
    'gk8SrPiAqlFm': 'Jane'},
   'created_at': '2025-03-30T19:31:44.461275+00:00',
   'updated_at': '2025-03-30T19:31:44.461275+00:00',
   'datatable_id': 'ddebc6ff-95c1-488b-91ca-b29adbe59a6c'},
  {'id': 'bdwIopRvFNI1',
   'data': {'id': 'bdwIopRvFNI1',
    'cCnutJRZqI9J': 'Rome',
    'eT0gFBhCCYIQ': 'What is the capital of Italy?',
    'gFJjoJ2m3uhZ': '3',
    'gk8SrPiAqlFm': 'John'},
   'created_at': '2025-03-30T19:31:45.011852+00:00',
   'updated_at': '

### Columns (for experiments)

In [ ]:
@patch
async def list_experiment_columns(
    self: RelayClient,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: t.Optional[str] = None,
    sort_dir: t.Optional[str] = None,
) -> t.Dict:
    """List columns in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request(
        "GET",
        f"projects/{project_id}/experiments/{experiment_id}/columns",
        params=params,
    )


@patch
async def get_experiment_column(
    self: RelayClient, project_id: str, experiment_id: str, column_id: str
) -> t.Dict:
    """Get a specific column in an experiment."""
    return await self._request(
        "GET", f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}"
    )


@patch
async def create_experiment_column(
    self: RelayClient,
    project_id: str,
    experiment_id: str,
    id: str,
    name: str,
    type: ColumnType,
    col_order: t.Optional[int] = None,
    settings: t.Optional[t.Dict] = None,
) -> t.Dict:
    """Create a new column in an experiment."""
    data = {"id": id, "name": name, "col_order": col_order, "type": type}
    if settings:
        data["settings"] = settings
    return await self._request(
        "POST",
        f"projects/{project_id}/experiments/{experiment_id}/columns",
        json_data=data,
    )


@patch
async def update_experiment_column(
    self: RelayClient,
    project_id: str,
    experiment_id: str,
    column_id: str,
    **column_data,
) -> t.Dict:
    """Update an existing column in an experiment."""
    return await self._request(
        "PATCH",
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}",
        json_data=column_data,
    )


@patch
async def delete_experiment_column(
    self: RelayClient, project_id: str, experiment_id: str, column_id: str
) -> None:
    """Delete a column from an experiment."""
    await self._request(
        "DELETE",
        f"projects/{project_id}/experiments/{experiment_id}/columns/{column_id}",
    )

In [ ]:
await client.create_experiment(TEST_PROJECT_ID, "New Experiment", "This is a new experiment")

{'id': '78fd6c58-7edf-4239-93d1-4f49185d8e49',
 'name': 'New Experiment',
 'description': 'This is a new experiment',
 'created_at': '2025-03-30T06:31:31.689269+00:00',
 'version_counter': 0,
 'project_id': 'e1b3f1e4-d344-48f4-a178-84e7e32e6ab6'}

In [ ]:
experiments = await client.list_experiments(TEST_PROJECT_ID)
EXPERIMENT_ID = experiments["items"][0]["id"]
EXPERIMENT_ID

'78fd6c58-7edf-4239-93d1-4f49185d8e49'

In [ ]:
for col in columns.values():
    await client.create_experiment_column(
        project_id=TEST_PROJECT_ID,
        experiment_id=EXPERIMENT_ID,
        id=col["id"],
        name=col["name"],
        type=col["type"],
)

Exception: API Error (400): Unknown error

### Rows (for experiments)

In [ ]:
@patch
async def list_experiment_rows(
    self: RelayClient,
    project_id: str,
    experiment_id: str,
    limit: int = 50,
    offset: int = 0,
    order_by: Optional[str] = None,
    sort_dir: Optional[str] = None,
) -> Dict:
    """List rows in an experiment."""
    params = {"limit": limit, "offset": offset}
    if order_by:
        params["order_by"] = order_by
    if sort_dir:
        params["sort_dir"] = sort_dir
    return await self._request(
        "GET", f"projects/{project_id}/experiments/{experiment_id}/rows", params=params
    )


@patch
async def get_experiment_row(
    self: RelayClient, project_id: str, experiment_id: str, row_id: str
) -> Dict:
    """Get a specific row in an experiment."""
    return await self._request(
        "GET", f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )


@patch
async def create_experiment_row(
    self: RelayClient, project_id: str, experiment_id: str, id: str, data: Dict
) -> Dict:
    """Create a new row in an experiment."""
    row_data = {"id": id, "data": data}
    return await self._request(
        "POST",
        f"projects/{project_id}/experiments/{experiment_id}/rows",
        json_data=row_data,
    )


@patch
async def update_experiment_row(
    self: RelayClient, project_id: str, experiment_id: str, row_id: str, data: Dict
) -> Dict:
    """Update an existing row in an experiment."""
    row_data = {"data": data}
    return await self._request(
        "PATCH",
        f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}",
        json_data=row_data,
    )


@patch
async def delete_experiment_row(
    self: RelayClient, project_id: str, experiment_id: str, row_id: str
) -> None:
    """Delete a row from an experiment."""
    await self._request(
        "DELETE", f"projects/{project_id}/experiments/{experiment_id}/rows/{row_id}"
    )